## Encoding Text

### Bag of words

In [1]:
vocab = {}  # maps word to integer representing it
word_encoding = 1
def bag_of_words(text):
    global word_encoding

    words = text.lower().split(" ")  # create a list of all of the words in the text, well assume there is no grammar in our text for this example
    bag = {}  # stores all of the encodings and their frequency

    for word in words:
        if word in vocab:
            encoding = vocab[word]  # get encoding from vocab
        else:
            vocab[word] = word_encoding
            encoding = word_encoding
            word_encoding += 1
    
        if encoding in bag:
            bag[encoding] += 1
        else:
            bag[encoding] = 1
  
    return bag

text = "this is a test to see if this test will work is is test a a"
bag = bag_of_words(text)
print(bag)
print(vocab)

{1: 2, 2: 3, 3: 3, 4: 3, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1}
{'this': 1, 'is': 2, 'a': 3, 'test': 4, 'to': 5, 'see': 6, 'if': 7, 'will': 8, 'work': 9}


In [2]:
positive_review = "I thought the movie was going to be bad but it was actually amazing"
negative_review = "I thought the movie was going to be amazing but it was actually bad"

pos_bag = bag_of_words(positive_review)
neg_bag = bag_of_words(negative_review)

print("Positive:", pos_bag)
print("Negative:", neg_bag)

Positive: {10: 1, 11: 1, 12: 1, 13: 1, 14: 2, 15: 1, 5: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1}
Negative: {10: 1, 11: 1, 12: 1, 13: 1, 14: 2, 15: 1, 5: 1, 16: 1, 21: 1, 18: 1, 19: 1, 20: 1, 17: 1}


### Integer Encoding

In [3]:
vocab = {}  
word_encoding = 1
def one_hot_encoding(text):
    global word_encoding

    words = text.lower().split(" ") 
    encoding = []  

    for word in words:
        if word in vocab:
            code = vocab[word]  
            encoding.append(code) 
        else:
            vocab[word] = word_encoding
            encoding.append(word_encoding)
            word_encoding += 1
  
    return encoding

text = "this is a test to see if this test will work is is test a a"
encoding = one_hot_encoding(text)
print(encoding)
print(vocab)

[1, 2, 3, 4, 5, 6, 7, 1, 4, 8, 9, 2, 2, 4, 3, 3]
{'this': 1, 'is': 2, 'a': 3, 'test': 4, 'to': 5, 'see': 6, 'if': 7, 'will': 8, 'work': 9}


In [4]:
positive_review = "I thought the movie was going to be bad but it was actually amazing"
negative_review = "I thought the movie was going to be amazing but it was actually bad"

pos_encode = one_hot_encoding(positive_review)
neg_encode = one_hot_encoding(negative_review)

print("Positive:", pos_encode)
print("Negative:", neg_encode)

Positive: [10, 11, 12, 13, 14, 15, 5, 16, 17, 18, 19, 14, 20, 21]
Negative: [10, 11, 12, 13, 14, 15, 5, 16, 21, 18, 19, 14, 20, 17]


### Word Embeddings

It attempts to not only encode the frequency and order of words but the meaning of those words in the sentence. It encodes each word as a dense vector that represents its context in the sentence.

Unlike the previous techniques word embeddings are learned by looking at many different training examples. You can add what's called an embedding layer to the beggining of your model and while your model trains your embedding layer will learn the correct embeddings for words. You can also use pretrained embedding layers.

This is the technique we will use for our examples and its implementation will be showed later on.

## Sentiment Analysis

In [5]:
from keras.datasets import imdb
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

In [6]:
train_data[1]

[1,
 194,
 1153,
 194,
 8255,
 78,
 228,
 5,
 6,
 1463,
 4369,
 5012,
 134,
 26,
 4,
 715,
 8,
 118,
 1634,
 14,
 394,
 20,
 13,
 119,
 954,
 189,
 102,
 5,
 207,
 110,
 3103,
 21,
 14,
 69,
 188,
 8,
 30,
 23,
 7,
 4,
 249,
 126,
 93,
 4,
 114,
 9,
 2300,
 1523,
 5,
 647,
 4,
 116,
 9,
 35,
 8163,
 4,
 229,
 9,
 340,
 1322,
 4,
 118,
 9,
 4,
 130,
 4901,
 19,
 4,
 1002,
 5,
 89,
 29,
 952,
 46,
 37,
 4,
 455,
 9,
 45,
 43,
 38,
 1543,
 1905,
 398,
 4,
 1649,
 26,
 6853,
 5,
 163,
 11,
 3215,
 10156,
 4,
 1153,
 9,
 194,
 775,
 7,
 8255,
 11596,
 349,
 2637,
 148,
 605,
 15358,
 8003,
 15,
 123,
 125,
 68,
 23141,
 6853,
 15,
 349,
 165,
 4362,
 98,
 5,
 4,
 228,
 9,
 43,
 36893,
 1157,
 15,
 299,
 120,
 5,
 120,
 174,
 11,
 220,
 175,
 136,
 50,
 9,
 4373,
 228,
 8255,
 5,
 25249,
 656,
 245,
 2350,
 5,
 4,
 9837,
 131,
 152,
 491,
 18,
 46151,
 32,
 7464,
 1212,
 14,
 9,
 6,
 371,
 78,
 22,
 625,
 64,
 1382,
 9,
 8,
 168,
 145,
 23,
 4,
 1690,
 15,
 16,
 4,
 1355,
 5,
 28,
 6,
 52,
 

making each review the same length

In [7]:
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

### Model

In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          2834688   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 28s 42ms/step - loss: 0.4162 - acc: 0.8134 - val_loss: 0.2907 - val_acc: 0.8830
Epoch 2/10
625/625 [==============================] - 24s 39ms/step - loss: 0.2392 - acc: 0.9076 - val_loss: 0.3908 - val_acc: 0.8524
Epoch 3/10
625/625 [==============================] - 23s 37ms/step - loss: 0.1859 - acc: 0.9304 - val_loss: 0.2684 - val_acc: 0.8874
Epoch 4/10
625/625 [==============================] - 23s 38ms/step - loss: 0.1507 - acc: 0.9460 - val_loss: 0.2925 - val_acc: 0.8858
Epoch 5/10
625/625 [==============================] - 27s 44ms/step - loss: 0.1281 - acc: 0.9545 - val_loss: 0.3223 - val_acc: 0.8838
Epoch 6/10
625/625 [==============================] - 24s 38ms/step - loss: 0.1113 - acc: 0.9615 - val_loss: 0.3199 - val_acc: 0.8900
Epoch 7/10
625/625 [==============================] - 25s 40ms/step - loss: 0.0974 - acc: 0.9666 - val_loss: 0.3856 - val_acc: 0.8854
Epoch 8/10
625/625 [==============================] - 26s 42ms

In [11]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 9s 11ms/step - loss: 0.6223 - acc: 0.8222
[0.6222668886184692, 0.8222000002861023]


In [12]:
word_index = imdb.get_word_index()

def encode_text(text):
    tokens = keras.preprocessing.text.text_to_word_sequence(text)
    tokens = [word_index[word] if word in word_index else 0 for word in tokens]
    return sequence.pad_sequences([tokens], MAXLEN)[0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)


[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0  12  17  13  4

In [13]:
reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
        if num != PAD:
            text += reverse_word_index[num] + " "

    return text[:-1]
  
print(decode_integers(encoded))

that movie was just amazing so amazing


In [14]:
def predict(text):
    encoded_text = encode_text(text)
    pred = np.zeros((1,250))
    pred[0] = encoded_text
    result = model.predict(pred) 
    print(result[0])

positive_review = "That movie was! really loved it and would great watch it again because it was amazingly great"
predict(positive_review)

negative_review = "that movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)

[0.85748625]
[0.38191178]


## RNN Play Generator

In [15]:
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

In [16]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [17]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [18]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



encoding

In [19]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
    return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [20]:
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [21]:
def int_to_text(ints):
    try:
        ints = ints.numpy()
    except:
        pass
    return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


creating training examples

In [22]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [23]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [24]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [25]:
for x, y in dataset.take(2):
    print("\n\nEXAMPLE\n")
    print("INPUT")
    print(int_to_text(x))
    print("\nOUTPUT")
    print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [26]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

### Model

In [27]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
      ])
    return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (64, None, 256)           16640     
                                                                 
 lstm_1 (LSTM)               (64, None, 1024)          5246976   
                                                                 
 dense_1 (Dense)             (64, None, 65)            66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


loss function

In [28]:
for input_example_batch, target_example_batch in data.take(1):
    example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [29]:
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[ 3.9832797e-03 -4.3352507e-04 -5.4502045e-04 ... -1.7849426e-04
   -6.6202017e-04 -3.9503782e-04]
  [ 5.8844378e-03 -5.7231011e-03 -1.6380940e-03 ...  1.2608948e-03
   -2.6878808e-03  4.5561413e-03]
  [ 2.5902861e-03  2.7363282e-04  2.7462770e-03 ...  3.2611808e-03
    2.2406138e-03  8.5719936e-03]
  ...
  [ 8.5322000e-03 -7.5981109e-03 -5.2043539e-04 ... -1.3035480e-03
    6.5418966e-03 -3.1635284e-03]
  [ 2.7310944e-03 -7.7114888e-03 -3.3030305e-03 ... -4.6551162e-03
    1.0336631e-02 -7.3442853e-04]
  [ 3.4876596e-03 -8.1091970e-03 -1.5885985e-03 ... -9.3047759e-03
    1.0606967e-02  3.3018803e-03]]

 [[-1.2054029e-03 -4.1164425e-03 -5.5347858e-03 ...  4.2412402e-03
    4.6510645e-03  6.8038161e-04]
  [-2.2224812e-03 -3.8065878e-03  1.1026461e-03 ...  5.7890778e-03
    5.7398221e-03  3.5987548e-03]
  [ 2.4411255e-03 -2.7729294e-03 -3.8660830e-05 ...  4.1681272e-03
    3.8985997e-03  1.6239621e-03]
  ...
  [ 4.9463706e-05 -4.8336666e-03 -4.2293807e-03 ...  2.5806227e

In [30]:
pred = example_batch_predictions[0]
print(len(pred))
print(pred)

100
tf.Tensor(
[[ 0.00398328 -0.00043353 -0.00054502 ... -0.00017849 -0.00066202
  -0.00039504]
 [ 0.00588444 -0.0057231  -0.00163809 ...  0.00126089 -0.00268788
   0.00455614]
 [ 0.00259029  0.00027363  0.00274628 ...  0.00326118  0.00224061
   0.00857199]
 ...
 [ 0.0085322  -0.00759811 -0.00052044 ... -0.00130355  0.0065419
  -0.00316353]
 [ 0.00273109 -0.00771149 -0.00330303 ... -0.00465512  0.01033663
  -0.00073443]
 [ 0.00348766 -0.0081092  -0.0015886  ... -0.00930478  0.01060697
   0.00330188]], shape=(100, 65), dtype=float32)


In [31]:
time_pred = pred[0]
print(len(time_pred))
print(time_pred)

65
tf.Tensor(
[ 3.9832797e-03 -4.3352507e-04 -5.4502045e-04 -3.6653306e-03
  3.8743597e-03 -1.3538413e-03 -8.0508937e-04  4.0859493e-05
  3.0082874e-03  1.9266575e-03  1.2154251e-02  5.5690240e-03
  3.4563898e-03 -7.5126952e-03 -5.5070780e-04 -2.4169432e-03
  3.5892772e-03  4.5903479e-03 -3.2490315e-03  1.1045387e-03
  2.1733386e-03  2.3194165e-03  1.7768067e-03 -4.2773336e-03
 -1.1283718e-04  5.4641762e-03 -5.0865929e-04  1.1294456e-03
  1.3249204e-04 -4.3496280e-03  8.4950868e-03  1.1413742e-03
  2.5721896e-03  2.3108658e-03 -6.7476742e-03 -1.4392820e-03
  1.0719163e-03  1.9055088e-03  4.0898609e-04  3.7891758e-03
  2.4635508e-04  1.2034554e-03 -2.3581746e-03  9.2050582e-03
  2.9189507e-03 -5.1036733e-03 -1.0822295e-03  6.8863910e-03
  1.1189389e-03 -1.2560295e-03 -4.7998637e-04 -4.6655945e-03
 -5.1624561e-04 -1.4578549e-03  2.5017196e-03  1.3079562e-03
  2.4401392e-03 -3.7029702e-03  1.9507530e-03  1.1097884e-03
 -1.1984726e-03  1.3170172e-03 -1.7849426e-04 -6.6202017e-04
 -3.950378

In [32]:
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars 

"jO!3eKNF$rjXhl?\n' sicH&zBp;NjHNDE$nIDejLhzKAjtxJCcWvf3MfZKuCRrpFdyx;v,qxYr\nhQgfWqHwBkm;gIyEh-$sgXCe!"

In [33]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [34]:
model.compile(optimizer='adam', loss=loss)

checkpoints

In [35]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

training

In [36]:
history = model.fit(data, epochs=3, callbacks=[checkpoint_callback])

Epoch 1/3
172/172 [==============================] - 312s 2s/step - loss: 2.5788
Epoch 2/3
172/172 [==============================] - 321s 2s/step - loss: 1.8796
Epoch 3/3
172/172 [==============================] - 331s 2s/step - loss: 1.6315


In [37]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [38]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [ ]:
# checkpoint_num = 2
# model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
# model.build(tf.TensorShape([1, None]))

generating text

In [39]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 800

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
    
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [40]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: hello
hellow it,
To be this envise the droud espict your grown takely own grace Is. I have laye as chaiter-tay's with mank:
So sweat me.

JULIET:
Now speak, have husberghands a woman,
Your lords Dord or thy brothern, fresh too counded by thee.

ABUSLY:
Sir, lowingst of the news, Nore and lives well spike.

JULIET:
A parcous simely word justless, hand I prighty.

AUHION
ENWER: did she sarmy far, ent you you fairt him
Ammanaligita and he parous.

JUDIET:
No make your'ers, Henry noves is mean unillon
Con the Vapricar's party, is counce,
I will marry place
Tull a looke furth with a hearper,
I'll counte a begt in the duken hath somes spill
That lad bace assiane that ame
Then deed sour fair, me: like apay; bust you
Proan the children of him and call as herewas;
Say, burds; receavesh me: undist: but come, t
